In [ ]:
# ======================================
# 360° ERP Image → Spherical Projection Pipeline
# Steps 1–5
# ======================================

# STEP 1: Add and display image
from google.colab import files
from IPython.display import Image as ColabImage
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Upload ERP image (e.g., Ocean360Picture.jpg)
uploaded = files.upload()
filename = list(uploaded.keys())[0]
ColabImage(filename)

# Read image in RGB format
erp = cv2.imread(filename)
erp = cv2.cvtColor(erp, cv2.COLOR_BGR2RGB)
H, W, _ = erp.shape

plt.figure(figsize=(10,5))
plt.imshow(erp)
plt.title("Step 1: Original ERP (Equirectangular Projection) Image")
plt.axis("off")
plt.show()

# ======================================
# STEP 2: Create 36x36 Matrix Placeholder
# ======================================
num_tiles = 36
tile_matrix = [[None for _ in range(num_tiles)] for _ in range(num_tiles)]
print(f"Step 2: Created {num_tiles}x{num_tiles} matrix for spherical tiles.")

# ======================================
# STEP 3: Reconstruct Image as Sphere
# ======================================
# Create meshgrid of longitude (λ) and latitude (φ)
u = np.linspace(0, W - 1, num_tiles)
v = np.linspace(0, H - 1, num_tiles)
U, V = np.meshgrid(u, v)

# Convert ERP pixel coords → spherical angles
lam = 2 * np.pi * (U / W - 0.5)       # longitude [-π, π]
phi = np.pi * (0.5 - V / H)           # latitude [π/2, -π/2]

# Convert spherical → Cartesian coords (X,Y,Z)
X = np.cos(phi) * np.cos(lam)
Y = np.sin(phi)
Z = np.cos(phi) * np.sin(lam)

# Sample colors from ERP for each spherical coordinate
U_int = np.clip(U.astype(int), 0, W - 1)
V_int = np.clip(V.astype(int), 0, H - 1)
colors = erp[V_int, U_int, :] / 255.0  # normalize to [0,1] for matplotlib

# Plot 3D sphere
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, facecolors=colors, rstride=1, cstride=1, linewidth=0, antialiased=False)
ax.set_box_aspect([1,1,1])
ax.set_title("Step 3: Seamless 360° Spherical Reconstruction (36×36 Grid)")
ax.axis("off")
plt.show()

# ======================================
# STEP 4: Split ERP into Spherical Tiles
# ======================================
delta_lambda = 2 * np.pi / num_tiles   # longitude step
delta_phi = np.pi / num_tiles          # latitude step

for j in range(num_tiles):
    for i in range(num_tiles):
        # Tile angular bounds
        lam_min = -np.pi + i * delta_lambda
        lam_max = -np.pi + (i + 1) * delta_lambda
        phi_max =  np.pi/2 - j * delta_phi
        phi_min =  np.pi/2 - (j + 1) * delta_phi

        # Convert angular bounds → ERP pixel coordinates
        u_min = int(((lam_min + np.pi) / (2 * np.pi)) * W)
        u_max = int(((lam_max + np.pi) / (2 * np.pi)) * W)
        v_min = int(((np.pi/2 - phi_max) / np.pi) * H)
        v_max = int(((np.pi/2 - phi_min) / np.pi) * H)

        # Clip to valid image range
        u_min, u_max = np.clip([u_min, u_max], 0, W)
        v_min, v_max = np.clip([v_min, v_max], 0, H)

        # Extract tile region from ERP
        tile_matrix[j][i] = erp[v_min:v_max, u_min:u_max, :]

print("Step 4: Image successfully divided into 36×36 spherical tiles.")

# Quick check: display one random tile
test_i, test_j = np.random.randint(0, num_tiles, 2)
tile = tile_matrix[test_j][test_i]
plt.figure(figsize=(4,4))
plt.imshow(tile)
plt.title(f"Sample Tile [{test_j}, {test_i}] (Lat/Long patch on sphere)")
plt.axis("off")
plt.show()

# ======================================
# STEP 5: Test Reconstruction (Rebuild ERP)
# ======================================
reconstructed = np.zeros_like(erp)
tile_h = H // num_tiles
tile_w = W // num_tiles

for j in range(num_tiles):
    for i in range(num_tiles):
        y0 = j * tile_h
        y1 = (j + 1) * tile_h
        x0 = i * tile_w
        x1 = (i + 1) * tile_w
        if tile_matrix[j][i] is not None:
            reconstructed[y0:y1, x0:x1, :] = cv2.resize(tile_matrix[j][i], (tile_w, tile_h))

plt.figure(figsize=(10,5))
plt.imshow(reconstructed)
plt.title("Step 5: Reconstructed ERP from 36×36 Spherical Tile Matrix")
plt.axis("off")
plt.show()
